In [1]:
!nvidia-smi

Thu Mar  7 00:55:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:4D:00.0 Off |                  Off |
| 30%   37C    P8              34W / 300W |      2MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### Installations

In [2]:
%%capture
!pip install -qU transformers \
    accelerate \
    hf_transfer \
    flash-attn \
    langchain \
    langchain_openai \
    langsmith

#### Environment Variables

In [3]:
import os
os.environ["HF_TOKEN"] = "<HF_TOKEN>"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["LANGCHAIN_API_KEY"] = "<LANGCHAIN_API_KEY>"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "7BSQLMaster"
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

#### Libraries

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
import torch
from time import sleep
from operator import itemgetter
from IPython.display import Markdown
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.schema import AIMessage, HumanMessage
from langchain.evaluation import load_evaluator
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tracers.context import tracing_v2_enabled
from langsmith import Client
from langchain.smith import RunEvalConfig, arun_on_dataset

In [7]:
def load_langchain_hugging_face_pipeline(model_name: str):

    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        attn_implementation="flash_attention_2",
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=model_name,
        rust_remote_code=True,
        padding_side="left"
    )

    pipeline_kwargs = {
        "max_new_tokens": 100,
        "do_sample": True,
        "temperature": 0.5,
        "top_k": 100,
        "top_p":0.90,
        "pad_token_id": tokenizer.pad_token_id if tokenizer.pad_token_id else tokenizer.eos_token_id
    }

    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        **pipeline_kwargs
    )

    return HuggingFacePipeline(pipeline=pipe)

### Generate questions and answers

#### Triplets {question, context, answer}

In [8]:
user = """Generate a set of question, context, and answer for querying a SQL database:

- **Question**: Pose a human query related to a specific task or information retrieval from a SQL database.
- **Context**: Describe the data table structure relevant to the question, using `CREATE TABLE` statements only.
- **Answer**: Provide the SQL query that effectively solves the human query, based on the given context.

Return a JSON object with `question`, `context`, `answer`.
"""

one_shot = """```json
{
  "question": "Find the total revenue generated from orders placed by customers in the USA for each product category in the year 2022.",
  "context": "CREATE TABLE Customers (\n  CustomerID INT PRIMARY KEY,\n  CustomerName VARCHAR(50),\n  Country VARCHAR(50)\n);\n\nCREATE TABLE Orders (\n  OrderID INT PRIMARY KEY,\n  CustomerID INT,\n  OrderDate DATE,\n  FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)\n);\n\nCREATE TABLE OrderDetails (\n  OrderDetailID INT PRIMARY KEY,\n  OrderID INT,\n  ProductID INT,\n  Quantity INT,\n  FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),\n  FOREIGN KEY (ProductID) REFERENCES Products(ProductID) \n);\n\nCREATE TABLE Products (\n  ProductID INT PRIMARY KEY,\n  ProductName VARCHAR(100),\n  CategoryID INT,\n  Price DECIMAL(10,2),\n  FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)\n);\n\nCREATE TABLE Categories (\n  CategoryID INT PRIMARY KEY,\n  CategoryName VARCHAR(50)\n);",
  "answer": "SELECT c.CategoryName, SUM(p.Price * od.Quantity) AS TotalRevenue\nFROM OrderDetails od\nJOIN Orders o ON od.OrderID = o.OrderID\nJOIN Customers cu ON o.CustomerID = cu.CustomerID\nJOIN Products p ON od.ProductID = p.ProductID  \nJOIN Categories c ON p.CategoryID = c.CategoryID\nWHERE cu.Country = 'USA' AND o.OrderDate BETWEEN '2022-01-01' AND '2022-12-31'\nGROUP BY c.CategoryName;"
}
```"""

In [9]:
human_template = HumanMessagePromptTemplate.from_template("{user_input}")
template = ChatPromptTemplate.from_messages([
    HumanMessage(content=user),
    AIMessage(content=one_shot),
    human_template
])
llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0.8,
)
parser = SimpleJsonOutputParser()
chain = template | llm | parser

In [10]:
# with tracing_v2_enabled(project_name="Triplets Generation"):
#     triplets = await chain.abatch([{"user_input": user}] * 10)

In [11]:
# len(triplets)
# triplets

#### Langsmith Dataset

In [12]:
client = Client()
dataset_name = "SQL Triplets - GPT4 Turbo"

In [13]:
if client.has_dataset(dataset_name=dataset_name):
    print("Dataset already exists")
else: 
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="Questions and answers to evaluate fine tuned SQL models.",
    )
    for triplet in triplets:
        client.create_example(
            inputs={k: v for k, v in triplet.items() if k != "answer"},
            outputs={k: v for k, v in triplet.items() if k == "answer"},
            dataset_id=dataset.id,
        )

Dataset already exists


### Evaluate Fine Tuned Models

In [14]:
template = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
`{question}`

### Context:
`{context}`

### Response:
"""
prompt = PromptTemplate.from_template(template=template)
initial_chain = (
    {"context": itemgetter("context"), "question": itemgetter("question")}
    | prompt 
)

In [15]:
evaluation_config = RunEvalConfig(
    evaluators=[
        RunEvalConfig.EmbeddingDistance(
            embeddings=OpenAIEmbeddings(model="text-embedding-3-large", dimensions=512),
            distance_metric="cosine",
            reference_key="answer",
            prediction_key="output",
            input_key="context" # Not sure about this one
        )
    ]
)

In [16]:
hf_owner = "jjovalle99/"
models = ["llama7b-ft-lora-sql-v2", "deci7b-ft-lora-sql-v2", "mistral7b-ft-lora-sql-v2", "gemma7b-ft-lora-sql-v2"]
models = [hf_owner + model for model in models]
names = ["Llama2", "DeciLM", "Mistral", "Gemma"]

In [17]:
for name, model_name in zip(names, models):

    print(f"Loading {name}...")
    hf_pipeline = load_langchain_hugging_face_pipeline(model_name=model_name)
    chain = initial_chain | hf_pipeline
    print(f"Load Complete!")

    print(f"Evaluating {name}...")
    await arun_on_dataset(
        dataset_name=dataset_name,
        llm_or_chain_factory=chain,
        evaluation=evaluation_config,
        client=client,
        project_name=f"7BSQLMaster - {name}",
        verbose=True
    )
    print(f"Evaluation complete...")

    del chain
    del hf_pipeline
    torch.cuda.empty_cache()
    sleep(10)
    

Loading Llama2...


config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

Load Complete!
Evaluating Gemma...
View the evaluation results for project '7BSQLMaster - Gemma' at:
https://smith.langchain.com/o/e1ff5e9a-fc1a-5ec0-91dc-86daf509e790/datasets/bc80884d-83c6-400e-a232-81cdb1553404/compare?selectedSessions=b72cfbd7-be15-4522-bf6f-a353ff756c41

View all tests for Dataset SQL Triplets - GPT4 Turbo at:
https://smith.langchain.com/o/e1ff5e9a-fc1a-5ec0-91dc-86daf509e790/datasets/bc80884d-83c6-400e-a232-81cdb1553404
[------------------------------------------------->] 10/10Evaluation complete...
